<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [328]:
import pandas as pd
import psycopg2

In [329]:
# вставьте сюда параметры подключения из юнита 1. Работа с базой данных из Python 

In [330]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

In [331]:
# чтобы постоянно не вылезало предупреждение
import warnings
warnings.filterwarnings('ignore', message = '.*SQLAlchemy connectable')

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [332]:
# текст запроса
query_3_1 = f'''
SELECT
       COUNT(id)
  FROM vacancies
'''

In [333]:
# результат запроса
pd.read_sql_query(query_3_1, connection).iloc[0, 0]

49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [334]:
# текст запроса
query_3_2 = f'''
SELECT
       COUNT(id)
  FROM employers
'''

In [335]:
# результат запроса
pd.read_sql_query(query_3_2, connection).iloc[0, 0]

23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [336]:
# текст запроса
query_3_3 = f'''
SELECT
       COUNT(id)
  FROM areas
'''

In [337]:
# результат запроса
pd.read_sql_query(query_3_3, connection).iloc[0, 0]

1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [338]:
# текст запроса
query_3_4 = f'''
SELECT
       COUNT(id)
  FROM industries
'''

In [339]:
# результат запроса
pd.read_sql_query(query_3_4, connection).iloc[0, 0]

294

***

### выводы по предварительному анализу данных:
Исходя из той немногочисленной информации, что была только что получена сложно сделать какие-то глубокие выводы.
На 23,5 тыс. работодателей приходится почти 50 тыс. вакансий (в среднем чуть больше 2 от одного работодателя, хотя, скорее всего, есть несколько крупных работодателей с большим количеством и остальные компании, разместившие всего 1) в более чем 1300 регионах. Размещённые вакасии охватывают почти 300 различных сфер деятельности.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [340]:
# текст запроса
query_4_1 = f'''
  SELECT
         a.name area,
         COUNT(v.area_id) AS cnt
    FROM vacancies AS v
         jOIN areas AS a /*соединяем таблицы с вакансиями и регионами*/
         ON a.id = v.area_id /*по id региона*/
GROUP BY a.name, v.area_id
ORDER BY 2 DESC
'''

In [341]:
# результат запроса
pd.read_sql_query(query_4_1, connection)

,area,cnt
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Шахтерск (Сахалинская область),1
765,Пестово,1
766,Пересвет,1
767,Первомайское (Ленинградская область),1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [342]:
# текст запроса
query_4_2 = f'''
SELECT
       COUNT(id)
  FROM vacancies
 WHERE salary_from IS NOT NULL 
    OR salary_to IS NOT NULL
'''

In [343]:
# результат запроса
pd.read_sql_query(query_4_2, connection).iloc[0, 0]

24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [344]:
# текст запроса
query_4_3 = f'''
SELECT
       ROUND(AVG(salary_from)) AS avg_from, /*ищем среднее значение зарплаты и округляем до целого*/
       ROUND(AVG(salary_to)) AS avg_to
  FROM vacancies
'''

In [345]:
# результат запроса
pd.read_sql_query(query_4_3, connection)

,avg_from,avg_to
0,71065.0,110537.0


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [346]:
# текст запроса
query_4_4 = f'''
  SELECT
         schedule,
         employment,
         COUNT(id)
    FROM vacancies
GROUP BY schedule, employment
ORDER BY 3 DESC
'''

In [347]:
# результат запроса
pd.read_sql_query(query_4_4, connection)

,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [348]:
# текст запроса
query_4_5 = f'''
  SELECT
         experience
    FROM vacancies
GROUP BY experience
ORDER BY COUNT(id) /*выводим столбец с опытом работы, но сортируем по количеству вакансий*/
'''

In [349]:
# результат запроса
pd.read_sql_query(query_4_5, connection)

,experience
0,Более 6 лет
1,Нет опыта
2,От 3 до 6 лет
3,От 1 года до 3 лет


***

### выводы по детальному анализу вакансий:
* Наибольшее количество вакансий сосредоточено в самых крупных городах.
* Почти у половины вакансий не указана зарплата.
* Количество вакансий на полнуый день и полную занятость выше, чем на все остальные сочетания рабочего графика и типов трудоустройства вместе взятых
* Зависимость количества вакансий от требуемого опыта работы вполне ожидаема.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [350]:
# текст запроса
query_5_1 = f'''
  SELECT 
         e.name
    FROM vacancies AS v
         JOIN employers AS e
         ON e.id = v.employer_id
GROUP BY v.employer_id, e.name
ORDER BY COUNT(v.id) DESC
'''

In [351]:
# результат запроса
pd.read_sql_query(query_5_1, connection).head(5)

,name
0,Яндекс
1,Ростелеком
2,Тинькофф
3,СБЕР
4,Газпром нефть


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [352]:
# текст запроса
query_5_2 = f'''
  SELECT
         a.name,
         COUNT(DISTINCT e.id) AS employer_cnt, 
         COUNT(DISTINCT v.id) AS vacancie_cnt 
    FROM areas AS a
         LEFT JOIN employers AS e /*соединяем таблицу регионов с таблицей работодателей*/
         ON a.id = e.area
         LEFT JOIN vacancies AS v /*и с таблицей вакансий*/
         ON a.id = v.area_id
GROUP BY a.id /*группируем по регионам, чтобы посчитать количество вакансий и работодателей*/
ORDER BY 3, 2 DESC
'''

In [353]:
# результат запроса
pd.read_sql_query(query_5_2, connection)

,name,employer_cnt,vacancie_cnt
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
1357,Алматы,721,1892
1358,Новосибирск,573,2006
1359,Минск,1115,2112
1360,Санкт-Петербург,2217,2851


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [354]:
# текст запроса
query_5_3 = f'''
  SELECT
         e.name,
         COUNT(DISTINCT v.area_id) AS area_cnt /*считаем только различные регионы, т.к. работодатель может публиковать несколько вакансий в одном*/
    FROM employers AS e
         LEFT JOIN vacancies AS v 
         ON e.id = v.employer_id
GROUP BY e.id
ORDER BY 2 DESC
'''

In [355]:
# результат запроса
pd.read_sql_query(query_5_3, connection)

,name,area_cnt
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [356]:
# текст запроса
query_5_4 = f'''
  SELECT
         COUNT(e.id)
    FROM employers AS e
         LEFT JOIN employers_industries AS ei 
         ON e.id = ei.employer_id
   WHERE ei.industry_id IS NULL
'''

In [357]:
# результат запроса
pd.read_sql_query(query_5_4, connection).iloc[0,0]

8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [358]:
# текст запроса
query_5_5 = f'''
  SELECT
         e.name
    FROM employers AS e
         LEFT JOIN employers_industries AS ei 
         ON e.id = ei.employer_id
GROUP BY e.id
  HAVING COUNT(ei.industry_id) = 4
ORDER BY e.name
OFFSET 2 LIMIT 1
'''

In [359]:
# результат запроса
pd.read_sql_query(query_5_5, connection).iloc[0,0]

'2ГИС'

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [360]:
# текст запроса
query_5_6 = f'''
  SELECT
         COUNT(e.id)
    FROM employers AS e /*соединяем таблицу работодателей с таблицей сфер деятельности, через таблицу работодатели/сферы деятельности*/
         JOIN employers_industries AS ei 
         ON e.id = ei.employer_id
         JOIN industries AS i 
         ON ei.industry_id = i.id
   WHERE i.name = 'Разработка программного обеспечения'
'''

In [361]:
# результат запроса
pd.read_sql_query(query_5_6, connection).iloc[0,0]

3553

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [362]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D0%BE%D0%B2'
million_cities = tuple(pd.read_html(url)[1]['Город'])

In [363]:
# текст запроса
query_5_7 = f'''
     WITH yandex_mc AS /*чтобы несколько раз не писать один и тот же запрос, пропишем здесь все соединения и условия в CTE*/
          (SELECT 
                  a.name,
                  COUNT(v.id) AS cnt
             FROM vacancies AS v
                  JOIN areas AS a 
                  ON a.id = v.area_id
                  JOIN employers AS e 
                  ON e.id = v.employer_id
            WHERE a.name IN {million_cities} 
              AND e.name = 'Яндекс'
         GROUP BY a.name)
         
   SELECT * 
     FROM yandex_mc
          
UNION ALL 
          
   SELECT 'Total', /*создаём дополнительную строку с помощью ручной генерации*/
          SUM(cnt) /*считаем сумму всех вакансий во всех городах-миллионниках*/
     FROM yandex_mc
'''

In [364]:
# результат запроса
pd.read_sql_query(query_5_7, connection)

,name,cnt
0,Волгоград,24.0
1,Воронеж,32.0
2,Екатеринбург,39.0
3,Казань,25.0
4,Краснодар,30.0
5,Красноярск,23.0
6,Москва,54.0
7,Нижний Новгород,36.0
8,Новосибирск,35.0
9,Омск,21.0


***

### выводы по анализу работодателей:
* Яндекс лидирует по количеству вакансий. В первую пятёрку также входят два банка, одна телекоммуникационная и одна ресурсодобывающая компании.
* Регионы, в которых нет вакансий, но много работодателей - это либо области, либо целые страны. Это означает, что вакансии в таблице указаны для отдельных населённых пунктов.
* Из компаний, у которых самое большое покрытие регионов вакансиями, две входят в пятёрку по количеству вакансий. Неожиданным представляется некий Поляков Денис Иванович. Какой-то индивидуальный предприниматель, судя по всему, разместил вакансии в 88 регионах. Возможно, конечно, что это просто множество небольших населённых пунктов внутри одного субъекта. Поэтому количество получается сопоставимым с Яндексом или Ростелекомом, которые, скорее всего, размещали вакансии только в областных или районных центрах.
* Более чем у трети работодателей не указана сфера деятельности.
* Около 15% компаний связаны с разработкой ПО.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [365]:
# текст запроса
query_6_1 = f'''
  SELECT
         COUNT(id)
    FROM vacancies
   WHERE LOWER(name) LIKE '%data%' 
      OR LOWER(name) LIKE '%данн%'
'''

In [366]:
# результат запроса
pd.read_sql_query(query_6_1, connection).iloc[0,0]

1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [367]:
# чтобы каждый раз не прописывать список сочетаний
# исключаем только 'ML', т.к. в нём нужен именно верхний регистр
ds_words = '{%data scientist%, %data science%, %исследователь данных%, %machine learning%, %машинн%обучен%}'

In [368]:
# текст запроса
query_6_2 = f'''
  SELECT
         COUNT(id)
    FROM vacancies
   WHERE (name ILIKE ANY ('{ds_words}') 
      OR (name LIKE '%ML%'
     AND name NOT ILIKE '%HTML%')) /*отдельное условие для 'ML', хотя здесь оно не влияет на вывод*/
     AND (name ILIKE '%junior%' 
      OR experience = 'Нет опыта' 
      OR employment = 'Стажировка')
'''

In [369]:
# результат запроса
pd.read_sql_query(query_6_2, connection).iloc[0,0]

51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [370]:
# текст запроса
query_6_3 = f'''
  SELECT
         COUNT(id)
    FROM vacancies
   WHERE (name ILIKE ANY ('{ds_words}') 
      OR (name LIKE '%ML%' /*отдельное условие для 'ML', хздесь оно влияет на вывод*/
     AND name NOT ILIKE '%HTML%')) 
     AND key_skills ~* 'SQL|postgres' /*ищем одну из двух подстрок без учёта регистра*/
'''

In [371]:
# результат запроса
pd.read_sql_query(query_6_3, connection).iloc[0,0]

201

4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [372]:
# текст запроса
query_6_4 = f'''
  SELECT
         COUNT(id)
    FROM vacancies
   WHERE (name ILIKE ANY ('{ds_words}') 
      OR (name LIKE '%ML%'
     AND name NOT ILIKE '%HTML%')) 
     AND key_skills ~* 'Python'
'''

In [373]:
# результат запроса
pd.read_sql_query(query_6_4, connection).iloc[0,0]

351

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [374]:
# текст запроса
query_6_5 = f'''
  SELECT
         ROUND(AVG(ARRAY_LENGTH(REGEXP_SPLIT_TO_ARRAY(key_skills, '\t'), 1)), 2) /*разбиваем строку на слова по табуляции, считаем кол-во элементов получившегося массива*/
    FROM vacancies
   WHERE (name ILIKE ANY ('{ds_words}') 
      OR (name LIKE '%ML%'
     AND name NOT ILIKE '%HTML%'))
'''

In [375]:
# результат запроса
pd.read_sql_query(query_6_5, connection).iloc[0,0]

6.41

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [376]:
# текст запроса
query_6_6 = f'''
  SELECT 
         experience,
         ROUND(AVG(COALESCE((salary_from + salary_to) / 2, salary_from, salary_to))) AS avg_salary
    FROM vacancies
   WHERE (name ILIKE ANY ('{ds_words}') 
      OR (name LIKE '%ML%'
     AND name NOT ILIKE '%HTML%'))
     AND (salary_from IS NOT NULL
      OR salary_to IS NOT NULL)
GROUP BY experience
'''

In [377]:
# результат запроса
pd.read_sql_query(query_6_6, connection)

,experience,avg_salary
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

### выводы по предметному анализу

* Из более чем 1700 вакансий имеет отношение к работе с данными, из них 480 относится непосредственно к data science, из которых 10% для начинающих
* Каждая 9-я вакансия требует знания SQL, а знания Python каждая 5-я
* В среднем более 6 ключевых навыков требуется в каждой вакансии, часто это несколько языков программирования, знание сред разработкки, операционных систем и различных концепций и методов
* Зарплатная вилка, в зависимости от опыта соискателя составляет в среднем от 75 тыс. руб. до 240 тыс. руб., разница более, чем в 3 раза

---

## Дополнительные исследования

доп. исследование 1

Посмотрим, скольким компаниям на рынке в принципе требуются специалисты по работе с данными

In [378]:
query_e_1 = f'''
  SELECT
         COUNT(DISTINCT e.id)
    FROM vacancies AS v
         JOIN employers AS e 
         ON e.id = v.employer_id
   WHERE (v.name ILIKE ANY ('{ds_words}') 
      OR (v.name LIKE '%ML%'
     AND v.name NOT ILIKE '%HTML%'))
'''

In [379]:
pd.read_sql_query(query_e_1, connection).iloc[0,0]

225

доп. исследование 2

Проверим, как связаны опыт и среднее количество требуемых навыков для ds

In [380]:
query_e_2 = f'''
  SELECT
         experience,
         AVG(ARRAY_LENGTH(REGEXP_SPLIT_TO_ARRAY(key_skills, '\t'), 1)) AS key_skill_cnt
    FROM vacancies
   WHERE (name ILIKE ANY ('{ds_words}') 
      OR (name LIKE '%ML%'
     AND name NOT ILIKE '%HTML%'))
GROUP BY experience
ORDER BY 2 DESC
'''

In [381]:
pd.read_sql_query(query_e_2, connection)

,experience,key_skill_cnt
0,Нет опыта,7.423077
1,От 1 года до 3 лет,6.731959
2,От 3 до 6 лет,5.988764
3,Более 6 лет,5.939394


доп. исследование 3

Посмотрим на средние зарплаты ds по регионам

In [382]:
query_e_3 = f'''
  SELECT 
         a.name,
         ROUND(AVG(COALESCE((salary_from + salary_to) / 2, salary_from, salary_to))) AS avg_salary
    FROM vacancies AS v
         JOIN areas AS a
         ON a.id = v.area_id
   WHERE (v.name ILIKE ANY ('{ds_words}') 
      OR (v.name LIKE '%ML%'
     AND v.name NOT ILIKE '%HTML%'))
     AND (v.salary_from IS NOT NULL
      OR v.salary_to IS NOT NULL)
GROUP BY a.id
ORDER BY 2 DESC
'''

In [383]:
pd.read_sql_query(query_e_3, connection)

,name,avg_salary
0,Кипр,300000.0
1,Армения,268863.0
2,Черногория,233794.0
3,Сербия,233794.0
4,Турция,233794.0
5,Москва,207630.0
6,Белгород,200000.0
7,Санкт-Петербург,173335.0
8,Новосибирск,166333.0
9,Рязань,136667.0


### выводы по дополнительному исследованию

* Все ds-вакансии представлены 225 компаниями
* Количество требуемых в вакансии навыков почему-то обратно (!) пропорционально требуемому опыту работы
* Зависимость средних зарплат от региона, в целом, ожидаемая, выделяется разве что Белгород, но это, скорее всего, связано с неполнотой данных о зарплатах

---

# Общие выводы по проекту

* Количество вакансий связанных с данными составляет чуть больше 3,5% от общего числа вакансий в базе. Непосредственно data science чуть меньше 1% от всех.
* Количество работодателей размещающих вакансии дата-сайентистов тоже чуть меньше 1% от всех компаний.
* Большинство вакансий сосредоточено в крупных городах и, в основном, размещается крупными компаниями. Они же размещают вакансиии в наибольшем количестве регионов, где у них расположены филиалы. Яндекс, в этом отношении, лидирует по нескольким показателям.
* Более чем у трети компаний не указана сфера деятельности, что накладывает ограничения на возможности исследования.
* В Data science почти в половине вакансий требуется знание SQL и почти в двух третях владение Python.
* Средняя зарплата довольно существенно растёт в зависимости от требуемого опыта, при этом количество требуемых навыков почему-то падает. Видимо, при отборе соискателей с большим опытом, HR уже не так сильно смотрят на владение конкретными навыками.
* Если брать только РФ, средние зарплаты, по-видимому, зависят от двух факторов - размер города и близость к Москве. Но из-за неполноты данных по зарплатам, сложно сказать наверняка
